In [3]:
import pandas as pd

codes= ['a','d','e','f']

dict(zip(range(0,len(codes)),codes))

{0: 'a', 1: 'd', 2: 'e', 3: 'f'}

In [5]:
from pathlib import Path

import typing as t
import re
import pandas as pd
import scipy
import yaml
import numpy as np
from loguru import logger
from tqdm import tqdm

import scipy.sparse

import sys
sys.path.append(".")
# from icd9 import ICD9


# try:
#     import format_data_for_training #script from auto-icd
# except ImportError:
#     # when running in a pytest context
import format_data_for_training

# input filepaths.
DIAGNOSIS_CSV_FP = "./data/mimiciii-14/DIAGNOSES_ICD.csv.gz"
PROCEDURES_CSV_FP = "./data/mimiciii-14/PROCEDURES_ICD.csv"
ICD9_DIAG_KEY_FP = "./data/mimiciii-14/D_ICD_DIAGNOSES.csv.gz"
ICD9_PROC_KEY_FP = "./data/mimiciii-14/D_ICD_PROCEDURES.csv"

# ICD_GEM_FP = "./data/ICD_general_equivalence_mapping.csv" #for conversion to ICd10

# output filepaths
XBERT_LABEL_MAP_FP = './data/intermediary-data/xbert_inputs/label_map.txt'
XBERT_TRAIN_RAW_TEXTS_FP = './data/intermediary-data/xbert_inputs/train_raw_texts.txt'
XBERT_VAL_RAW_TEXTS_FP = './data/intermediary-data/xbert_inputs/val_raw_texts.txt'
XBERT_TEST_RAW_TEXTS_FP = './data/intermediary-data/xbert_inputs/test_raw_texts.txt'

XBERT_Y_TRN_FP = './data/intermediary-data/xbert_inputs/Y.trn.npz'
XBERT_Y_VAL_FP = './data/intermediary-data/xbert_inputs/Y.val.npz'
XBERT_Y_TST_FP = './data/intermediary-data/xbert_inputs/Y.tst.npz'
DF_TRAIN_FP ='./data/intermediary-data/df_train.pkl'
DF_TEST_FP = './data/intermediary-data/df_test.pkl'


def main():
    with open('params.yaml', 'r') as f:
        params = yaml.safe_load(f.read())
    icd_version_specified = str(params['prepare_for_xbert']['icd_version'])
    diag_or_proc_param = params['prepare_for_xbert']['diag_or_proc']
    assert diag_or_proc_param == 'proc' or diag_or_proc_param == 'diag', 'Must specify either \'proc\' or \'diag\'.'
    note_category_param = params['prepare_for_xbert']['note_category']
    icd_seq_num_param = params['prepare_for_xbert']['one_or_all_icds']
    subsampling_param = params['prepare_for_xbert']['subsampling']

    logger.info(f'Using ICD version {icd_version_specified}...')
    assert icd_version_specified == '9' or icd_version_specified == '10', 'Must specify one of ICD9 or ICD10.'
    logger.info('Reformatting raw data with subsampling {}', 'enabled' if subsampling_param else 'disabled')

    df_train, df_val, df_test = \
        format_data_for_training.construct_datasets(
            diag_or_proc_param, note_category_param, subsampling_param)


    # X_trn = xbert_prepare_txt_inputs(df_train, 'training')
    # X_val = xbert_prepare_txt_inputs(df_val, 'validation')
    # X_tst = xbert_prepare_txt_inputs(df_test, 'testing')


    icd_labels, desc_labels = xbert_create_label_map(icd_version_specified, diag_or_proc_param)
    #need codes, not descriptions right now.

    icd_labels.to_csv('data/icd_labels.csv')

    data_train = xbert_prepare_Y_maps(
        df_train, icd_labels.tolist(), icd_version_specified)

    data_val = xbert_prepare_Y_maps(
        df_val, icd_labels.tolist(), icd_version_specified)

    data_test = xbert_prepare_Y_maps(
        df_test, icd_labels.tolist(), icd_version_specified)

    logger.info('Created dataframes of hospital admission to assigned ICD codes.')

    data_train = data_train.insert(loc=0,column='TEXT',value= df_train['TEXT'])
    data_val  = data_val.insert(loc=0,column='TEXT',value= df_val['TEXT'])
    data_test = data_test.insert(loc=0,column='TEXT',value= df_test['TEXT'])


    logger.info(
        'Done preprocessing for multilabel classification. Saving CSV of columns [text, code1,code2,...,codeL].'
    )

ModuleNotFoundError: No module named 'format_data_for_training'